In [ ]:
import faultier
import serial
import time

from tqdm.notebook import trange
ft = faultier.Faultier()
lp = faultier.LiveMarkerPlotOld()

ser = serial.Serial(ft.get_serial_path(), baudrate=115200)
ser.timeout = 1
ft.configure_glitcher(
    power_cycle_output = faultier.OUT_MUX0,
    power_cycle_length = 300000,
    trigger_source = faultier.TRIGGER_IN_EXT0,
    trigger_type = faultier.TRIGGER_PULSE_POSITIVE,
    glitch_output = faultier.OUT_CROWBAR
)

ft.power_cycle()
print(ser.read(3))

success = []
fails = []
detected = []

for d in trange(0, 100000, 100):
    for p in range(0, 700, 10):
        if ser.in_waiting:
            ser.read(ser.in_waiting)
        ft.glitch(delay=d, pulse=p)
        data = ser.read(3)
        # print(data)
        try:
            if b"Q" in data:
                print(f"Successful glitch - Delay: {d} Pulse: {p}")
                success.append([d, p])
                lp.update([success, detected])
            elif b"G" in data:
                print(f"Glitch detector triggered - Delay: {d} Pulse: {p}")
                detected.append([d, p])
                lp.update([success, detected])
                # time.sleep(5)
            else:
                fails.append([d, p])
        except:
            fails.append([d, p])